# {mod}`fabric`

```{topic} 依赖 paramiko

参考：[Paramiko](https://www.paramiko.org/index.html)

{mod}`paramiko` 是 SSHv2 协议的纯 python(3.6+) 实现，提供客户端和服务器功能。它为高级 SSH 库 {mod}`fabric` 提供了基础，建议您在运行远程 shell 命令或传输文件等常见客户端用例中使用 Fabric。
```

文档见：[Fabric](https://docs.fabfile.org/en/latest/index.html)

## 通过 `Connection` 和 `run` 运行命令

Fabric 最基本的用途是通过 SSH 在远程系统上执行 shell 命令，然后（可选地）查询结果。默认情况下，远程程序的输出直接打印到您的终端，并被捕获。

`Connection` 代表 SSH 连接，并提供 Fabric API 的核心，比如 `run`。连接对象至少需要一个主机名才能成功创建，并且可以通过用户名和/或端口号进一步参数化。你可以通过 `args/kwargs` 显式地给出这些参数：

```python
from fabric import Connection

Connection(host='web1', user='deploy', port=2202)
```

或者通过在 `host` 参数中填充 ` [user@]host[:port]` 字符串（尽管这纯粹是为了方便；无论何时出现歧义，都要使用 `kwargs`！）：

```python
Connection('deploy@web1:2202')
```


{class}`~fabric.connection.Connection` 对象的方法（如 `run`）通常返回 {class}`invoke.runners.Result` （或其子类）的实例，暴露了上面看到的各种细节：请求了什么，远程操作发生时发生了什么，以及最终结果是什么。

```{note}
通过使用 `connect_kwargs` 参数，可以将许多低级 SSH 连接参数（如私钥和超时）直接提供给 SSH 后端。
```

示例：

```python
>>> from fabric import Connection
>>> c = Connection('web1')
>>> result = c.run('uname -s')
Linux
>>> result.stdout.strip() == 'Linux'
True
>>> result.exited
0
>>> result.ok
True
>>> result.command
'uname -s'
>>> result.connection
<Connection host=web1>
>>> result.connection.host
'web1'
```

## 通过自动响应获得超级用户特权

需要以远程系统的超级用户身份运行程序？您可以通过 `run` 调用 `sudo` 程序，并且（如果您的远程系统没有配置无密码 `sudo`）手动响应密码提示，如下所示。（注意需要请求一个远程伪终端；否则，大多数 `sudo` 实现在密码提示时都会变得暴躁。）

```python
>>> from fabric import Connection
>>> c = Connection('db1')
>>> c.run('sudo useradd mydbuser', pty=True)
[sudo] password:
<Result cmd='sudo useradd mydbuser' exited=0>
>>> c.run('id -u mydbuser')
1001
<Result cmd='id -u mydbuser' exited=0>
```

每次手工输入密码会让人老去；值得庆幸的是，Invoke 强大的命令执行功能包括使用预定义输入自动响应程序输出的能力。我们可以在 `sudo` 中使用这个：

```python
>>> from invoke import Responder
>>> from fabric import Connection
>>> c = Connection('host')
>>> sudopass = Responder(
    pattern=r'\[sudo\] password:',
    response='mypassword\n',
)
>>> c.run('sudo whoami', pty=True, watchers=[sudopass])
[sudo] password:
root
<Result cmd='sudo whoami' exited=0>
```

很难在代码片段中显示，但是当上面的代码被执行时，用户不需要输入任何东西，密码被自动发送到远程程序。

## SSH 身份认证

即使在“普通”OpenSSH客户端中，对远程服务器的身份验证也涉及多个潜在的秘密和配置源；Fabric 不仅支持其中的大多数，而且有更多自己的功能。

```{note}
由于 Fabric 本身试图不重新发明太多的 Paramiko 功能，所以大多数时候配置身份验证值归结为“如何为 SSHClient 设置关键字参数值”。，这又意味着在 `connect_kwargs` [配置](https://docs.fabfile.org/en/latest/concepts/configuration.html)子树或 `Connection` 的 `connect_kwargs` 关键字参数中设置值。
```

### 私钥文件

存储在磁盘上的私钥可能是 SSH 最常见的验证机制。Fabric 提供了多种方法来配置要使用的路径，其中大多数最终合并成一个路径列表交给 `SSHClient.connect(key_filename=[...])`，顺序如下：

- 如果在 `Connection` 的 `connect_kwargs` 参数中存在 `key_filename` 密钥，则它们首先出现在列表中。（这基本上是非cli用户的“运行时”选项。）
- 配置设置 `connect_kwargs.key_filename` 可以通过多种方式设置（根据[配置文档](https://docs.fabfile.org/en/latest/concepts/configuration.html)），包括通过 `--identity` CLI标志（设置配置的覆盖级别；因此，当使用此标志时，来自其他配置源的关键文件名值将被覆盖。）这个值在整个列表中排名第二。
- 使用带有 `IdentityFile` 指令的 `ssh_config` 文件可以让你与其他 SSH 客户端共享配置；这些值排在最后。

### 加密密码

如果您的私钥文件是通过密码短语保护的，它可以通过几种方式提供：

- `connect_kwargs.passphrase` 配置选项是提供要自动使用的密码短语的最直接方法。
    ```{note}
    对这种类型的数据使用实际的磁盘配置文件并不总是明智的，但请记住，[配置系统](https://docs.fabfile.org/en/latest/concepts/configuration.html) 能够从其他来源加载数据，例如 shell 环境甚至任意远程数据库。
    ```
- 如果你喜欢在运行时手动输入密码，可以使用命令行选项 `--prompt-for-passphrase`，这让 Fabric 在进程开始时以交互方式提示用户，并将输入的值存储在 `connect_kwargs.passphrase` 中（在 `overrides` 级别）。

### 私钥对象

实例化自己的 [PKey](https://docs.paramiko.org/en/latest/api/keys.html#paramiko.pkey.PKey) 对象并将其放入 `connect_kwargs.pkey` 中。就是这样！如果要加载的 key 内容（这些类可以从文件路径或字符串中加载）是加密的，那么就需要负责处理密码。

### SSH 代理

默认情况下（类似于 OpenSSH 的行为方式）Paramiko 将尝试连接到正在运行的 SSH 代理（Unix 风格，例如 live `SSH_AUTH_SOCK`，或者 Windows 上的 Pageant）。这可以通过设置 `connect_kwargs.allow_agent` 为 `False` 来禁用。

### 密码

密码验证相对简单：

- 可以直接配置 `connect_kwargs.password`；
- 如果您希望在会话开始时提示输入，请指定 `--prompt-for-login-password`。